# Understanding the relationship/s between MEVs

**Third Stage** Running multiple linear regression just like in *Quarterly_selected_countries_secondPlan_corr_Multi_LinearReg_DT* but this time on all metrics. i.e. we will be changing y and use all other features as x

**Import Libraries**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as st



from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LinearRegression

# Libraries:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

**Upload datasets**

In [ ]:
mev = pd.read_csv('MEV_Selected_top_countries.csv',sep=";")

In [ ]:
mev.head()

,Date,Korea_Nominal_GDP,Korea_CPI,Korea_Unemployment_rate,Korea_Residential_property_index,US_Nominal_GDP,US_CPI,US_Unemployment_rate,US_Residential_property_index,Canada_Nominal_GDP,...,Switzerland_Residential_property_index,UK_Nominal_GDP,UK_CPI,UK_Unemployment_rate,UK_Residential_property_index,D_Nominal_GDP,D_CPI,D_Unemployment_rate,D_Residential_property_index,Unnamed: 25
0,1975-01-01,2370600,9.012579,NaN,63.2476,1616.116,52.566667,8.27,60.02,167572,...,36.2205,26777,14.043801,3.969991,5.4978,NaN,39.142725,3.6,128.1556,NaN
1,1975-04-01,2583900,9.738219,NaN,65.3178,1651.853,53.200000,8.87,60.96,173748,...,35.7642,28090,15.371077,4.339689,5.6187,NaN,39.903834,4.2,127.2398,NaN
2,1975-07-01,2690300,10.384638,NaN,67.5589,1709.820,54.266667,8.47,61.16,182328,...,35.4102,29219,16.046223,4.699658,5.7999,NaN,40.208277,4.3,126.3735,NaN
3,1975-10-01,2898800,10.810701,NaN,69.3324,1761.831,55.266667,8.30,62.22,188532,...,35.2497,30850,16.598615,5.071648,5.8603,NaN,40.512721,4.2,125.2582,NaN
4,1976-01-01,3275500,11.030391,NaN,69.9437,1820.487,55.900000,7.73,62.86,196288,...,34.4582,32405,17.197039,5.298913,5.9812,NaN,41.186845,4.1,123.4281,NaN


**General cleaning**

In [ ]:
mev = mev.iloc[:,:-1]
mev['Date'] = pd.to_datetime(mev['Date'], format='%Y-%m-%d')

mev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Date                                    188 non-null    datetime64[ns]
 1   Korea_Nominal_GDP                       188 non-null    int64         
 2   Korea_CPI                               188 non-null    float64       
 3   Korea_Unemployment_rate                 128 non-null    float64       
 4   Korea_Residential_property_index        188 non-null    float64       
 5   US_Nominal_GDP                          188 non-null    float64       
 6   US_CPI                                  188 non-null    float64       
 7   US_Unemployment_rate                    188 non-null    float64       
 8   US_Residential_property_index           188 non-null    float64       
 9   Canada_Nominal_GDP                      188 non-null  

In [ ]:
mev.isnull().sum()

Date                                       0
Korea_Nominal_GDP                          0
Korea_CPI                                  0
Korea_Unemployment_rate                   60
Korea_Residential_property_index           0
US_Nominal_GDP                             0
US_CPI                                     0
US_Unemployment_rate                       0
US_Residential_property_index              0
Canada_Nominal_GDP                         0
Canada_CPI                                 0
Canada_Unemployment_rate                   0
Canada_Residential_property_index          0
Switzerland_Nominal_GDP                   20
Switzerland_CPI                            0
Switzerland_Unemployment_rate              0
Switzerland_Residential_property_index     0
UK_Nominal_GDP                             0
UK_CPI                                     0
UK_Unemployment_rate                       0
UK_Residential_property_index              0
D_Nominal_GDP                             64
D_CPI     

**3 columns are missing data due to reporting time horizon difference**

In [ ]:
impute = SimpleImputer(strategy='mean')

mev_imputed = pd.DataFrame(impute.fit_transform(mev.iloc[:,1:]), columns=mev.iloc[:,1:].columns)
mev_imputed['Date'] = mev['Date']
mev_imputed.head()

,Korea_Nominal_GDP,Korea_CPI,Korea_Unemployment_rate,Korea_Residential_property_index,US_Nominal_GDP,US_CPI,US_Unemployment_rate,US_Residential_property_index,Canada_Nominal_GDP,Canada_CPI,...,Switzerland_Residential_property_index,UK_Nominal_GDP,UK_CPI,UK_Unemployment_rate,UK_Residential_property_index,D_Nominal_GDP,D_CPI,D_Unemployment_rate,D_Residential_property_index,Date
0,2370600.0,9.012579,3.509115,63.2476,1616.116,52.566667,8.27,60.02,167572.0,21.991046,...,36.2205,26777.0,14.043801,3.969991,5.4978,619115.241935,39.142725,3.6,128.1556,1975-01-01
1,2583900.0,9.738219,3.509115,65.3178,1651.853,53.200000,8.87,60.96,173748.0,22.517777,...,35.7642,28090.0,15.371077,4.339689,5.6187,619115.241935,39.903834,4.2,127.2398,1975-04-01
2,2690300.0,10.384638,3.509115,67.5589,1709.820,54.266667,8.47,61.16,182328.0,23.255201,...,35.4102,29219.0,16.046223,4.699658,5.7999,619115.241935,40.208277,4.3,126.3735,1975-07-01
3,2898800.0,10.810701,3.509115,69.3324,1761.831,55.266667,8.30,62.22,188532.0,23.729260,...,35.2497,30850.0,16.598615,5.071648,5.8603,619115.241935,40.512721,4.2,125.2582,1975-10-01
4,3275500.0,11.030391,3.509115,69.9437,1820.487,55.900000,7.73,62.86,196288.0,24.071635,...,34.4582,32405.0,17.197039,5.298913,5.9812,619115.241935,41.186845,4.1,123.4281,1976-01-01


In [ ]:
mev_imputed.isnull().sum()

Korea_Nominal_GDP                         0
Korea_CPI                                 0
Korea_Unemployment_rate                   0
Korea_Residential_property_index          0
US_Nominal_GDP                            0
US_CPI                                    0
US_Unemployment_rate                      0
US_Residential_property_index             0
Canada_Nominal_GDP                        0
Canada_CPI                                0
Canada_Unemployment_rate                  0
Canada_Residential_property_index         0
Switzerland_Nominal_GDP                   0
Switzerland_CPI                           0
Switzerland_Unemployment_rate             0
Switzerland_Residential_property_index    0
UK_Nominal_GDP                            0
UK_CPI                                    0
UK_Unemployment_rate                      0
UK_Residential_property_index             0
D_Nominal_GDP                             0
D_CPI                                     0
D_Unemployment_rate             

# Multiple Regression

**Predicting macroeconomic metric by other MEVs**

Below is scoring metrics and coefficients

In [ ]:
test_size = 38

results = dict()

def run_reg(df):

  for i in df.columns:
    y = df[i]
    X = df.drop(columns=i)
    print(f"Predicting {i} by other MEVs:\n\n")
    X_train = X[:-test_size]
    X_test = X[-test_size:]

    y_train = y[:-test_size]
    y_test = y[-test_size:]
    
    # setting pipeline
    pipe = Pipeline(steps = [("standard_scale", StandardScaler()),
                            ("modeling", LinearRegression())])

    pipe.fit(X_train, y_train)

    pred = pipe.predict(X_test)
    MAE = metrics.mean_absolute_error(y_test, pred)
    MSE = metrics.mean_squared_error(y_test, pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, pred))
    R2 = metrics.r2_score(y_test, pred)

    results[i]= R2
    # Metrics
    print('MAE', MAE)
    print('MSE', MSE)
    print('RMSE', RMSE )
    print('R2 Score', R2 )
    print('='*1000)
  



run_reg(mev_imputed.drop(columns='Date')) 

#results

res = pd.DataFrame(results.items(), columns=['Predicted','R2'])

Predicting Korea_Nominal_GDP by other MEVs:


MAE 26227242.22701006
MSE 1019412216005823.6
RMSE 31928235.403883874
R2 Score 0.5421702300256781
Predicting Korea_CPI by other MEVs:


MAE 7.998280757302385
MSE 88.22095573412315
RMSE 9.392601116523748
R2 Score -7.086414056435373
Predicting Korea_Unemployment_rate by other MEVs:


MAE 4.456019613150711
MSE 30.03515201611137
RMSE 5.4804335609613375
R2 Score -304.1866956484954
Predicting Korea_Residential_property_index by other MEVs:


MAE 41.55665009199945
MSE 2042.846250943545
RMSE 45.197856707409755
R2 Score -53.81309554137446
Predicting US_Nominal_GDP by other MEVs:


MAE 306.66456588219245
MSE 195155.74013336445
RMSE 441.76434909730375
R2 Score 0.9516619604106221
Predicting US_CPI by other MEVs:


MAE 5.185040217300422
MSE 37.15668267809928
RMSE 6.095628161075713
R2 Score 0.7547085619159725
Predicting US_Unemployment_rate by other MEVs:


MAE 1.2383386931765026
MSE 2.3775673464895486
RMSE 1.5419362329517874
R2 Score 0.31892170680516096


In [ ]:
res.sort_values(by='R2', ascending=False)

,Predicted,R2
4,US_Nominal_GDP,0.951662
7,US_Residential_property_index,0.927574
16,UK_Nominal_GDP,0.908707
17,UK_CPI,0.838978
11,Canada_Residential_property_index,0.819644
5,US_CPI,0.754709
8,Canada_Nominal_GDP,0.693782
20,D_Nominal_GDP,0.675450
0,Korea_Nominal_GDP,0.542170
15,Switzerland_Residential_property_index,0.502328


#End